## Fase 3 --- Calibração dos modelos e retrospectiva

Aconteceu que a biblioteca sklearn tem uma função que automaticamente calibra os parâmetros da regressão linear regularizada. Por isso, essa fase deve fugir um pouco do esperado. Fica a lição para projetos futuros.

#### Passo 1

>aplicar todos os modelos que restaram como candidatos da fase passada.

Só tomei a regressão linear regularizada (Ridge) como opção na fase passada. Ela será aplicada abaixo. Utiliza-se o código Python gerado nas fases anteriores.

In [1]:
from src.preprocessing import *

In [2]:
from sklearn.linear_model import RidgeCV

In [3]:
from numpy import logspace

# Not sure the alphas we should try, so just use a logarithmic
#   distributed space.
tryAlphas = logspace(-5, 5, 10)
model = RidgeCV(alphas=tryAlphas, cv=10)

In [4]:
filtPricesDf = fetchFiltPricesDf(pricesDf);

/usr/lib/python3/dist-packages/pandas/core/frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [5]:
X_train, X_test, y_train, y_test = splitTrainTest(filtPricesDf);

In [6]:
model.fit(X_train, y_train);

In [ ]:
predictions = model.predict(X_test)

In [12]:
def calcSSE(ground_truth, y_hat):
    return ((ground_truth - y_hat) ** 2).sum()

def calcTSS(ground_truth):
    mu = ground_truth.mean()
    return ((ground_truth - mu) ** 2).sum()
            
def calcR2Score(ground_truth, y_hat):
    sse = calcSSE(ground_truth, y_hat)
    tss = calcTSS(ground_truth)
    rss = tss - sse
    return rss / tss

r2_score = calcR2Score(y_test, predictions)

print(r2_score)

0.9999762815205986


#### Passo 2

Não foi necessário fazer qualquer coisa nesse passo, pois a biblioteca proveu meios de fazer isso automaticamente. O parâmetro de regularização ideal encontrado para o modelo é dado abaixo.

In [13]:
model.alpha_

0.2782559402207126

#### Passo 3

Só agora a pouco é que se percebeu que um erro crasso foi cometido na criação do modelo de mineração de dados.

Ao invés de se criar um preditor, criou-se um regressor. Isto é, ao invés de, com base nos dados até um certo dia, prever os atributos do próximo dia, ou algo do tipo, o modelo criado toma como entrada os atributos independentes e dá como saída o atributo dependente, notavelmente "ocMean".

Como regressor, obviamente teve sucesso, pois conseguiu generalizar para o conjunto de teste com cerca de 0.002% de erro, com base na pontuação R2.